### **In this study, I will apply Text Cleaning on Amazon Reviews **



---



---



In [1]:
import pandas as pd
import numpy as np
import re
import string
!pip install emot --upgrade
import emot
!pip install emoji --upgrade
import emoji
from bs4 import BeautifulSoup
!pip install contractions --upgrade
import contractions
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
!pip install pyspellchecker --upgrade
from spellchecker import SpellChecker

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('output/Amazon_reviews_processed_2.csv')
df.head()

,review_id,star_rating,review_date,review,sentiment_score,review_category
0,RZGFXZ2HYHHRA,5,2015-05-02,CHOOSE THE RIGHT COLORING PENCILS This is th...,positive,product
1,R2I37K23W0YCC9,5,2014-11-02,"Great Taste, A Little Complicated I'll start b...",positive,product
2,RPMW8FVG5H0ZW,5,2012-10-26,Beyond Science and Religion...Endorsement and ...,positive,product
3,R35SXKTW1A8AWO,5,2014-04-24,A truly unique work for this generation EDIT: ...,positive,product
4,R1KA9YCQ31E3C0,5,2013-08-14,Stop kissing frogs 🐸 🐸 🐸 🐸 I've been trying Pa...,positive,product


In [3]:
cols = ['review_id',  'review_date',]
df.drop(cols, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582135 entries, 0 to 582134
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   star_rating      582135 non-null  int64 
 1   review           582135 non-null  object
 2   sentiment_score  582135 non-null  object
 3   review_category  582135 non-null  object
dtypes: int64(1), object(3)
memory usage: 17.8+ MB


In [4]:
text = df.review[4]
text

'Stop kissing frogs 🐸 🐸 🐸 🐸 I\'ve been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I\'ve kissed a lot of frogs.<br /><br />One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for.  First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes.  The author posted an errata on her website that was just as confusing as the mistakes in the cookbook.  I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.<br /><br />I also got the Wheat Belly Cookbook that\'s loaded with truly awful recipes, including the basic bread recipe that went 

-------------

# **Text cleaning techniques** 

#### Parsing html to Text - removes URLS, HTML elements, tags

In [5]:
soup = BeautifulSoup(text, 'html.parser')
text=soup.get_text()
text

'Stop kissing frogs 🐸 🐸 🐸 🐸 I\'ve been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I\'ve kissed a lot of frogs.One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for.  First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes.  The author posted an errata on her website that was just as confusing as the mistakes in the cookbook.  I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.I also got the Wheat Belly Cookbook that\'s loaded with truly awful recipes, including the basic bread recipe that went directly into the trash 

#### Converting Emojis and Emoticons
***What is the difference between emoji and emoticons?***

*   :-) is an emoticon
*   😜 → emoji.

In [6]:
# converting emoji to text using emoji package
text =emoji.demojize(text, delimiters=("", ""))
text

'Stop kissing frogs frog frog frog frog I\'ve been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I\'ve kissed a lot of frogs.One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for.  First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes.  The author posted an errata on her website that was just as confusing as the mistakes in the cookbook.  I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.I also got the Wheat Belly Cookbook that\'s loaded with truly awful recipes, including the basic bread recipe that went directly int

In [7]:
# converting emoticon to text using emot package
def emoticon_to_text(text):
  res=""
  emot_obj = emot.core.emot()
  for word in text.split():
    emoticons_dict=emot_obj.emoticons(word)
    if emoticons_dict['flag']:
      mean=emoticons_dict['mean'][0].split()
      word="_".join(mean)
    res=res+word+" "
  return res

text=emoticon_to_text(text)
text

'Stop kissing frogs frog frog frog frog I\'ve been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I\'ve kissed a lot of frogs.One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for. First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes. The author posted an errata on her website that was just as confusing as the mistakes in the cookbook. I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.I also got the Wheat Belly Cookbook that\'s loaded with truly awful recipes, including the basic bread recipe that went directly into t

#### Removing Contractions
There are so many contractions in the text we type so to expand them we will use the contractions library.

For example, the text “I’ll eat pizza” and if we remove the punctuations them the text will look like this “I ll will eat pizza”. Here, “I ll” doesn’t hold any information to the text that’s why we use the contraction.

In [8]:
text = contractions.fix(text)
text

'Stop kissing frogs frog frog frog frog I have been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I have kissed a lot of frogs.One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for. First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes. The author posted an errata on her website that was just as confusing as the mistakes in the cookbook. I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.I also got the Wheat Belly Cookbook that is loaded with truly awful recipes, including the basic bread recipe that went directly into

#### Lower Casing the Data
From the first glance we just lower case the data. The idea is to convert the input text into the same casing format so that it converts ‘DATA’, ’Data’, ‘DaTa’, ‘DATa’ into ‘data’.

In [9]:
text=text.lower()
text

'stop kissing frogs frog frog frog frog i have been trying paleo/grain-free cookbooks since i first discovered this healthy way of eating earlier this year. . .and i have kissed a lot of frogs.one sold here on amazon [[asin:0578116405 satisfying eats: grain free, sugar free & hunger free cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, i think) that i fell for. first i bought the kindle edition (for my ipad) that was so full of quirks and glitches that it was practically useless, so i ordered the hard copy only to learn that it (1st edition) was loaded with mistakes. the author posted an errata on her website that was just as confusing as the mistakes in the cookbook. i should have sent it back. the few things i did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.i also got the wheat belly cookbook that is loaded with truly awful recipes, including the basic bread recipe that went directly into

#### remove text in square brackets, 

In [10]:
# reference https://stackoverflow.com/a/14603508
def remove_text_inside_brackets(text, brackets="()[]"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    return ''.join(saved_chars)

In [11]:
text = remove_text_inside_brackets(text)
text

'stop kissing frogs frog frog frog frog i have been trying paleo/grain-free cookbooks since i first discovered this healthy way of eating earlier this year. . .and i have kissed a lot of frogs.one sold here on amazon  has a strange warm-and-fuzzy cult following and an undeserved rating  that i fell for. first i bought the kindle edition  that was so full of quirks and glitches that it was practically useless, so i ordered the hard copy only to learn that it  was loaded with mistakes. the author posted an errata on her website that was just as confusing as the mistakes in the cookbook. i should have sent it back. the few things i did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.i also got the wheat belly cookbook that is loaded with truly awful recipes, including the basic bread recipe that went directly into the trash after cutting the first slice. i sent this one back  because it was nothing special and hopelessly repetitive.however, i have

#### Removing Punctuations and Numbers
The second most common text processing technique is removing punctuations from the textual data. The punctuation removal process will help to treat each text equally. For example, the word data and data! are treated equally after the process of removal of punctuations. <br> <br>
string.punctuation in python contains these symbols !"#$%&\'()*+,-./:;?@[\\]^_{|}~`

In [12]:
# text="".join([i if i not in string.punctuation else " " for i in text ])

# Remove punctuations and numbers
text = re.sub('[^a-zA-Z]', ' ', text)
text

'stop kissing frogs frog frog frog frog i have been trying paleo grain free cookbooks since i first discovered this healthy way of eating earlier this year     and i have kissed a lot of frogs one sold here on amazon  has a strange warm and fuzzy cult following and an undeserved rating  that i fell for  first i bought the kindle edition  that was so full of quirks and glitches that it was practically useless  so i ordered the hard copy only to learn that it  was loaded with mistakes  the author posted an errata on her website that was just as confusing as the mistakes in the cookbook  i should have sent it back  the few things i did make from the cookbook where an utter disaster  especially the bread muffin cookie things i also got the wheat belly cookbook that is loaded with truly awful recipes  including the basic bread recipe that went directly into the trash after cutting the first slice  i sent this one back  because it was nothing special and hopelessly repetitive however  i have

#### Spelling Correction
One another important text preprocessing step is spelling correction. Typos are common in text data and we might want to correct those spelling mistakes before we do our analysis.<br>

If we are interested in wrinting a spell corrector of our own, we can probably start with famous code from Peter Norvig.<br>

In this notebook, let us use the python package pyspellchecker for our spelling correction.<br>

In [13]:
def correct_spellings(text):
  spell = SpellChecker()
  corrected_text = []
  misspelled_words = spell.unknown(text.split())
  for word in text.split():
      if word in misspelled_words:
          corrected_text.append(spell.correction(word))
      else:
          corrected_text.append(word)
  return " ".join(corrected_text)

In [14]:
sen = "thnks for readin the notebook"
sen =correct_spellings(sen)
sen

'thanks for reading the notebook'

In [15]:
text = correct_spellings(text)
text

'stop kissing frogs frog frog frog frog i have been trying paleo grain free cookbooks since i first discovered this healthy way of eating earlier this year and i have kissed a lot of frogs one sold here on amazon has a strange warm and fuzzy cult following and an undeserved rating that i fell for first i bought the kindle edition that was so full of quirks and glitches that it was practically useless so i ordered the hard copy only to learn that it was loaded with mistakes the author posted an errand on her website that was just as confusing as the mistakes in the cookbook i should have sent it back the few things i did make from the cookbook where an utter disaster especially the bread muffin cookie things i also got the wheat belly cookbook that is loaded with truly awful recipes including the basic bread recipe that went directly into the trash after cutting the first slice i sent this one back because it was nothing special and hopelessly repetitive however i have finally found my 

#### Removal of Stop Words

In [16]:
def stop_words_remover(text):
  stop = stopwords.words('english')
  return " ".join(x for x in text.split() if x not in stop)

text = stop_words_remover(text)
text

'stop kissing frogs frog frog frog frog trying paleo grain free cookbooks since first discovered healthy way eating earlier year kissed lot frogs one sold amazon strange warm fuzzy cult following undeserved rating fell first bought kindle edition full quirks glitches practically useless ordered hard copy learn loaded mistakes author posted errand website confusing mistakes cookbook sent back things make cookbook utter disaster especially bread muffin cookie things also got wheat belly cookbook loaded truly awful recipes including basic bread recipe went directly trash cutting first slice sent one back nothing special hopelessly repetitive however finally found prince cookbook gem author world famous sandwich bread new benchmark grain free everyday sandwich loaves close wheat flour bread tasted could happily live rest life eating slow cooker sesame orange chicken coconut lime rice best kindle version ipad works perfectly quirks glitches every recipe picture even index thumbnail picture 

In [17]:
# Single character removal
text = re.sub(r"\s+[a-zA-Z]\s+", '', text)

# Removing multiple spaces
# text = re.sub(r'\s+', ' ', sentence)
text

'stop kissing frogs frog frog frog frog trying paleo grain free cookbooks since first discovered healthy way eating earlier year kissed lot frogs one sold amazon strange warm fuzzy cult following undeserved rating fell first bought kindle edition full quirks glitches practically useless ordered hard copy learn loaded mistakes author posted errand website confusing mistakes cookbook sent back things make cookbook utter disaster especially bread muffin cookie things also got wheat belly cookbook loaded truly awful recipes including basic bread recipe went directly trash cutting first slice sent one back nothing special hopelessly repetitive however finally found prince cookbook gem author world famous sandwich bread new benchmark grain free everyday sandwich loaves close wheat flour bread tasted could happily live rest life eating slow cooker sesame orange chicken coconut lime rice best kindle version ipad works perfectly quirks glitches every recipe picture even index thumbnail picture 

#### Removal of Frequent words
In the previos preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us.<br>

So this step is to remove the frequent words in the given corpus. If we use something like tfidf, this is automatically taken care of.<br>

Let us get the most common words and then remove them <br>

In [18]:
def remove_freqwords(text):
  cnt = Counter()
  for word in text.split():
    cnt[word] += 1
  cnt.most_common(10)
  FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
  return " ".join([word for word in str(text).split() if word not in FREQWORDS])

text = remove_freqwords(text)
text

'stop kissing frogs trying paleo grain free cookbooks since discovered healthy way eating earlier year kissed lot frogs sold amazon strange warm fuzzy cult following undeserved rating fell bought kindle edition full quirks glitches practically useless ordered hard copy learn loaded mistakes author posted errand website confusing mistakes sent back things make utter disaster especially muffin cookie things also got wheat belly loaded truly awful recipes including basic went directly trash cutting slice sent back nothing special hopelessly repetitive however finally found prince gem author world famous new benchmark grain free everyday loaves close wheat flour tasted could happily live rest life eating slow cooker sesame orange chicken coconut lime rice best kindle version ipad works perfectly quirks glitches every picture even index thumbnail picture every single paleo make tip better looking loaf world famous perfect pan bake small square instead wide shallow rectangular sure size perf

#### Removal of Rare words
This is very similar to previous preprocessing step but we will remove the rare words from the corpus.

In [19]:
def remove_rarewords(text):
    cnt = Counter()
    n_rare_words = 10
    RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

text= remove_rarewords(text)
text

'stop kissing frogs trying paleo grain free cookbooks since discovered healthy way eating earlier year kissed lot frogs sold amazon strange warm fuzzy cult following undeserved rating fell bought kindle edition full quirks glitches practically useless ordered hard copy learn loaded mistakes author posted errand website confusing mistakes sent back things make utter disaster especially muffin cookie things also got wheat belly loaded truly awful recipes including basic went directly trash cutting slice sent back nothing special hopelessly repetitive however finally found prince gem author world famous new benchmark grain free everyday loaves close wheat flour tasted could happily live rest life eating slow cooker sesame orange chicken coconut lime rice best kindle version ipad works perfectly quirks glitches every picture even index thumbnail picture every single paleo make tip better looking loaf world famous perfect pan bake small square instead wide shallow rectangular sure size perf

#### Lemmatization & Stemming

##### Stemming
Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form (From Wikipedia)<br>

For example, if there are two words in the corpus walks and walking, then stemming will stem the suffix to make them walk. But say in another example, we have two words console and consoling, the stemmer will remove the suffix and make them consol which is not a proper english word.<br>

There are several type of stemming algorithms available and one of the famous one is porter stemmer which is widely used. We can use nltk package for the same.<br>

In [20]:
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

print(stem_words(text))

stop kiss frog tri paleo grain free cookbook sinc discov healthi way eat earlier year kiss lot frog sold amazon strang warm fuzzi cult follow undeserv rate fell bought kindl edit full quirk glitch practic useless order hard copi learn load mistak author post errand websit confus mistak sent back thing make utter disast especi muffin cooki thing also got wheat belli load truli aw recip includ basic went directli trash cut slice sent back noth special hopelessli repetit howev final found princ gem author world famou new benchmark grain free everyday loav close wheat flour tast could happili live rest life eat slow cooker sesam orang chicken coconut lime rice best kindl version ipad work perfectli quirk glitch everi pictur even index thumbnail pictur everi singl paleo make tip better look loaf world famou perfect pan bake small squar instead wide shallow rectangular sure size perfect pan bake small squar instead wide shallow rectangular sure size


We can see that words like trying and since have tranformed to root words due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

##### Lemmatization
Lemmatization is similar to stemming in reducing inflected words to their word stem but differs in the way that it makes sure the root word (also called as lemma) belongs to the language.<br>

As a result, this one is generally slower than stemming process. So depending on the speed requirement, we can choose to use either stemming or lemmatization.<br>

Let us use the WordNetLemmatizer in nltk to lemmatize our sentences.<br>

lemmatization process depends on the POS tag to come up with the correct lemma.So we also need to provide the POS tag of the word along with the word for lemmatizer in nltk. Depending on the POS, the lemmatizer may return different results.<br>

Let us take the example, stripes and check the lemma when it is both verb and noun.<br>

In [21]:
lemmatizer = WordNetLemmatizer()

print("Word is : running")
print("stemmer result : ",stemmer.stem("running"))
print("Lemma result : ",lemmatizer.lemmatize("running"))
print("Lemma result for verb : ",lemmatizer.lemmatize("running", 'v'))
print("Lemma result for noun : ",lemmatizer.lemmatize("running", 'n'))

Word is : running
stemmer result :  run
Lemma result :  running
Lemma result for verb :  run
Lemma result for noun :  running


In [22]:
def lemmatize_words(text):
    wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

text= lemmatize_words(text)
text

'stop kiss frog try paleo grain free cookbook since discover healthy way eat early year kiss lot frog sell amazon strange warm fuzzy cult follow undeserved rating fell bought kindle edition full quirk glitch practically useless order hard copy learn load mistake author post errand website confuse mistake send back thing make utter disaster especially muffin cookie thing also get wheat belly load truly awful recipe include basic go directly trash cut slice send back nothing special hopelessly repetitive however finally find prince gem author world famous new benchmark grain free everyday loaves close wheat flour taste could happily live rest life eat slow cooker sesame orange chicken coconut lime rice best kindle version ipad work perfectly quirks glitch every picture even index thumbnail picture every single paleo make tip well look loaf world famous perfect pan bake small square instead wide shallow rectangular sure size perfect pan bake small square instead wide shallow rectangular s

In [23]:
def apply_all_text_cleaners(text):
    text = BeautifulSoup(text, 'html.parser').get_text() #parse html
    text = emoji.demojize(text, delimiters=("", "")) # emoji to text
    text = emoticon_to_text(text) #emoticons to text
    text = contractions.fix(text) # replace contractions: i'll to i will
    text = text.lower() # normalising text to lower case
    text = remove_text_inside_brackets(text) 
    text = re.sub('[^a-zA-Z]', ' ', text) # Removing Punctuations and Numbers
    text = stop_words_remover(text)
    text = re.sub(r"\s+[a-zA-Z]\s+", '', text) # Single character removal
    text = remove_freqwords(text)
    text= remove_rarewords(text)
    text= lemmatize_words(text)
    return text


In [24]:
test = 'Stop kissing frogs 🐸 🐸 🐸 🐸 I\'ve been trying Paleo/grain-free cookbooks since I first discovered this healthy way of eating earlier this year. . .and I\'ve kissed a lot of frogs.<br /><br />One sold here on Amazon [[ASIN:0578116405 Satisfying Eats: Grain Free, Sugar Free & Hunger Free Cookbook]] has a strange warm-and-fuzzy cult following and an undeserved rating (from her groupies, I think) that I fell for.  First I bought the Kindle edition (for my iPad) that was so full of quirks and glitches that it was practically useless, so I ordered the hard copy only to learn that it (1st edition) was LOADED with mistakes.  The author posted an errata on her website that was just as confusing as the mistakes in the cookbook.  I should have sent it back. The few things I did make from the cookbook where an utter disaster, especially the bread/muffin/cookie things.<br /><br />I also got the Wheat Belly Cookbook that\'s loaded with truly awful recipes, including the basic bread recipe that went directly into the trash after cutting the first slice.  I sent this one back [[ASIN:1936608863 Make it Paleo: Over 200 Grain Free Recipes For Any Occasion]] because it was nothing special and hopelessly repetitive.<br /><br />However, I\'ve finally found my prince.  This cookbook is a gem.  The author\'s \\\\"World-Famous Sandwich Bread\\\\" is the new benchmark for all grain-free everyday sandwich loaves - it\'s as close to wheat flour bread as any I\'ve tasted.  And I could happily live the rest of my life eating only \\\\"Slow Cooker Sesame-Orange Chicken\\\\" over \\\\"Coconut-Lime Rice\\\\" (cauliflower rice like no other - you\'ll be fooled).  Best of all, the Kindle version on my iPad works perfectly - no quirks or glitches.  Every recipe has a picture, and even the index has a thumbnail picture of every single recipe.<br /><br />If you get only one Paleo cookbook, make it this one.<br /><br />PS -- A tip from me for a better-looking loaf of the \\\\"World-Famous Sandwich Bread\\\\" ==> [[ASIN:B0017Z0EW6 Fat Daddio\'s Anodized Aluminum Bread Pan, 7.75 Inch x 3.75 Inch x 2.75 Inch]] is the PERFECT pan to bake the bread in - you\'ll get small square slices, instead of wide/shallow rectangular slices.  Be sure to get the 7.75-inch x 3.75-inch x 2.75-inch size.7.75 Inch x 3.75 Inch x 2.75 Inch]] is the PERFECT pan to bake the bread in - you\'ll get small square slices, instead of wide/shallow rectangular slices.  Be sure to get the 7.75-inch x 3.75-inch x 2.75-inch size.'

In [25]:
test = apply_all_text_cleaners(test)
test

'stop kiss frog try paleo grain free cookbook since discover healthy way eat early year kiss lot frog sell amazon strange warm fuzzy cult follow undeserved rating fell bought kindle edition full quirk glitch practically useless order hard copy learn load mistake author post errata website confuse mistake send back thing make utter disaster especially muffin cookie thing also get wheat belly load truly awful recipe include basic go directly trash cut slice send back nothing special hopelessly repetitive however finally find prince gem author world famous new benchmark grain free everyday loaves close wheat flour taste could happily live rest life eat slow cooker sesame orange chicken coconut lime rice best kindle version ipad work perfectly quirks glitch every picture even index thumbnail picture every single paleo make p tip well look loaf world famous perfect pan bake small square instead wide shallow rectangular sure size perfect pan bake small square instead wide shallow rectangular

In [26]:
print(text==test)

False


In [27]:
df['Text'] = df['review'].apply(lambda x: apply_all_text_cleaners(x))

C:\Users\Yogesh\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [29]:
df.to_csv('output/Amazon_reviews_cleaned.csv', index=False)

In [30]:
df

,star_rating,review,sentiment_score,review_category,Text
0,5,CHOOSE THE RIGHT COLORING PENCILS This is th...,positive,product,choose right list item use beautiful intricate...
1,5,"Great Taste, A Little Complicated I'll start b...",positive,product,great taste start say book amazing world good ...
2,5,Beyond Science and Religion...Endorsement and ...,positive,product,beyond religion endorsement support another su...
3,5,A truly unique work for this generation EDIT: ...,positive,product,unique edit point ridiculous literally purchas...
4,5,Stop kissing frogs 🐸 🐸 🐸 🐸 I've been trying Pa...,positive,product,stop kiss frog try paleo grain free cookbook s...
...,...,...,...,...,...
582130,3,It's ok I haven't yet found a cookbook for my ...,neutral,product,want eat
582131,3,Book is in great condition I have several Al-a...,neutral,product,chance really plan put read
582132,3,Achieving Career Success on your Terms It's a ...,neutral,product,woman white corporation iwould recommend book ...
582133,3,good reference got book to have an updated IM ...,neutral,product,detail yet


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582135 entries, 0 to 582134
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   star_rating      582135 non-null  int64 
 1   review           582135 non-null  object
 2   sentiment_score  582135 non-null  object
 3   review_category  582135 non-null  object
 4   Text             582135 non-null  object
dtypes: int64(1), object(4)
memory usage: 22.2+ MB


In [33]:
df["star_rating"].value_counts()

5    197774
4    181205
1     78520
3     75613
2     49023
Name: star_rating, dtype: int64

In [34]:
df.isna().sum()

star_rating        0
review             0
sentiment_score    0
review_category    0
Text               0
dtype: int64

In [35]:
df[['review','Text']].head()

,review,Text
0,CHOOSE THE RIGHT COLORING PENCILS This is th...,choose right list item use beautiful intricate...
1,"Great Taste, A Little Complicated I'll start b...",great taste start say book amazing world good ...
2,Beyond Science and Religion...Endorsement and ...,beyond religion endorsement support another su...
3,A truly unique work for this generation EDIT: ...,unique edit point ridiculous literally purchas...
4,Stop kissing frogs 🐸 🐸 🐸 🐸 I've been trying Pa...,stop kiss frog try paleo grain free cookbook s...
